In [1]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
p_masks = Path('/data/datasets/DAVIS/DAVIS2016/Annotations/1080p')

for split in ['val', 'train']:
    flat_anns = []
    anns = []
    with open(f'/data/datasets/DAVIS/DAVIS2016/ImageSets/1080p/{split}.txt', 'r') as f:
        all_anns = f.readlines()
    for train in all_anns:
        anns.append(train.strip().split(' ')[1].split('/')[-2])
    categories = list(set(anns))

    for category in categories:
        ow, oh = 0, 0
        p_mask = p_masks / category
        masks = list(p_mask.glob('*.png'))
        rt = []
        for idx, mask in enumerate(masks):
            
            img = cv2.imread(str(mask), cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(str(mask))
            
            if idx == 0:
                oh, ow = img.shape
            
            # Thresholding to ensure it's a binary mask (if not already binary)
            _, binary_img = cv2.threshold(img, 1, 255, cv2.THRESH_BINARY)
            
            # Find contours in the binary mask
            contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if len(contours) > 0:
                # Initialize variables to store the min/max coordinates
                x_min, y_min = float('inf'), float('inf')
                x_max, y_max = 0, 0
                
                # Loop through all contours to find the overall bounding box
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    
                    # Update the minimum and maximum coordinates
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x + w)
                    y_max = max(y_max, y + h)
                
                # Calculate the final bounding box coordinates (top-left corner and width/height)
                final_x = x_min
                final_y = y_min
                final_w = x_max - x_min
                final_h = y_max - y_min
                
                rt_dict = {
                    'fno': idx,
                    'x': final_x,
                    'y': final_y,
                    'w': final_w,
                    'h': final_h
                }
                
                rt.append(rt_dict)
                
                # if category == 'bear':
                #     # Convert the image to RGB for visualization
                #     # output_img = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
                    
                #     # Draw the single bounding box on the output image (in green with a thickness of 2)
                #     cv2.rectangle(img2, (final_x, final_y), (final_x + final_w, final_y + final_h), (0, 255, 0), 2)
                    
                #     # Plot the result using matplotlib
                #     plt.figure(figsize=(10, 10))
                #     plt.imshow(img2)
                #     plt.axis('off')
                #     plt.show()
                
                # print(f"Final bounding box: x={final_x}, y={final_y}, width={final_w}, height={final_h}")
        
        sample = {
                'video_uid': category,
                'clip_uid': category,
                'annotation_uid': category,
                'query_set': category,
                'clip_fps': 5,
                'clip_duration': len(masks) / 5,
                'original_width': ow,
                'original_height': oh,
                'query_frame': len(masks) - 1,
                'object_title': category,
                'visual_crop': rt[0],
                'response_track_valid_range': [rt[0]['fno'], rt[-1]['fno']],
                'response_track': rt,
                    }
        flat_anns.append(sample)

    with open(f'/data/soyeonhong/vq2d/vq2d-lightning/data/DAIVS2016_{split}.json', 'w') as f:
        f.write(json.dumps(flat_anns))
            

In [3]:
ow, oh

(1920, 1080)